[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/OshanJayawardana/biometrics-self-supervised/blob/main/supervised_gait/gait.ipynb)

In [ ]:
! wget "https://data.mendeley.com/api/datasets-v2/datasets/fwhn8hmz4f/zip/download?version=2" -c -O "gait.zip"
! unzip "gait.zip" -d "gait_dataset/"
! rm "gait.zip"

In [ ]:
! pip install tensorflow_addons

In [1]:
import os
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split

import tensorflow as tf
import tensorflow_addons as tfa
from tensorflow.keras import Input, Model
from tensorflow.keras.layers import Conv1D, Dense, MaxPooling1D, Concatenate, BatchNormalization, ReLU, Add, GlobalAveragePooling1D, GlobalMaxPooling1D, Dropout

from backbones import *

In [ ]:
frame_size = 50
x_train=np.array([])
y_train=[]
count=0
path = "/content/gait_dataset/IDNet's dataset/user_coordinates"
for user_id in range(1,51):
  for session_id in range(1,16):
    try:
      filename = "u"+str(user_id).rjust(3, '0')+"_w"+str(session_id).rjust(3, '0')+"_data_user_coord.csv"
      data = pd.read_csv(os.path.join(path,filename))
      data = np.array(data)
      data=np.lib.stride_tricks.sliding_window_view(data, (frame_size,data.shape[1]))[::frame_size//2, :]
      data=data.reshape(data.shape[0],data.shape[2],data.shape[3])
      count+=data.shape[0]
      if x_train.shape[0]==0:
        x_train  = data
        y_train += [user_id-1]*data.shape[0]
      else:
        x_train  = np.concatenate((x_train,data), axis=0)
        y_train += [user_id-1]*data.shape[0]
    except FileNotFoundError:
      continue

In [ ]:
def norma(x_all):
  x = np.reshape(x_all,(x_all.shape[0]*x_all.shape[1],x_all.shape[2]))
  scaler = StandardScaler()
  x = scaler.fit_transform(x)
  x_all = np.reshape(x,(x_all.shape[0],x_all.shape[1],x_all.shape[2]))
  x=[]
  return x_all

In [ ]:
x_train = norma(x_train)

In [ ]:
x_train, x_test, y_train, y_test = train_test_split(x_train, np.array(y_train), test_size=0.4)
x_test, x_val, y_test, y_val = train_test_split(x_test, y_test, test_size=0.5)

In [ ]:
ks = 3
con =4
num_classes=50
inputs = Input(shape=(50,3))
x = Conv1D(filters=16*con,kernel_size=ks,strides=1, padding='same')(inputs)
x = BatchNormalization()(x)
x = ReLU()(x)
x = MaxPooling1D(pool_size=4, strides=4)(x)
x = Dropout(rate=0.1)(x)
x = resnetblock(x, CR=32*con, KS=ks)
x = resnetblock(x, CR=32*con, KS=ks)
x = resnetblock(x, CR=64*con, KS=ks)
x = resnetblock(x, CR=64*con, KS=ks)
x = resnetblock(x, CR=128*con, KS=ks)
x = resnetblock(x, CR=128*con, KS=ks)
x = resnetblock_final(x, CR=128*con, KS=ks)
x = Dense(64, activation='relu')(x)
x = Dense(32, activation='relu')(x)
outputs = Dense(num_classes, activation='softmax')(x)
resnettssd = Model(inputs, outputs)

callback = tf.keras.callbacks.EarlyStopping(monitor='val_accuracy', restore_best_weights=True, patience=5)
lr_schedule = tf.keras.optimizers.schedules.ExponentialDecay(initial_learning_rate = 0.0001, decay_rate=0.5, decay_steps=1000000)# 0.0001, 0.9, 100000
optimizer = tf.keras.optimizers.Adam(learning_rate=lr_schedule)
resnettssd.compile(optimizer=optimizer, loss='sparse_categorical_crossentropy', metrics=['accuracy'] )
history = resnettssd.fit(x_train, y_train, validation_data=(x_val, y_val), epochs=100, callbacks=callback, batch_size=128)

results = resnettssd.evaluate(x_test,y_test)
print("test acc:", results[1])

#Calculating kappa score
metric = tfa.metrics.CohenKappa(num_classes=num_classes, sparse_labels=True)
metric.update_state(y_true=y_test , y_pred=resnettssd.predict(x_test))
result = metric.result()
print('kappa score: ',result.numpy())